In [ ]:
import re
import os

In [22]:
class Query:
  type : str
  conditions : list
  result : str | None
  '''
    Input Format:
    -> Ask : [CONDITIONS] ?
      Example => A, B, C ?
    -> Add : [CONDITIONS] > [RESULT]
      Example => A, B, C > D
  '''
  def __init__ (self, input: str):
    try:
      assert "?" in input or ">" in input
      ask_pattern = r'^([\w\s\-\.$\(\)]+(,\s?[\w\s\-\.$\(\)]+)*)\?$'
      add_pattern = r'^([\w\s\-\.$\(\)]*(,\s?[\w\s\-\.$\(\)]+)*)\s?>\s?[\w\s\-\.$\(\)]+$'
      input = input.strip()

      if (bool(re.match(ask_pattern, input))):
        self.type = "ask"
        self.conditions = input.split("?")[0].split(",")
        self.conditions = [condition.strip() for condition in self.conditions]
        self.result = None
        
      elif (bool(re.match(add_pattern, input))):
        self.type = "add"
        self.conditions = input.split(">")[0].split(",")
        self.conditions = [condition.strip() for condition in self.conditions if len(condition) > 0]
        self.result = input.split(">")[1].strip()
        assert self.result is not None, "None result value"

      else:
        raise ValueError("Invalid string pattern")
      
    except Exception as e:
      print(f"[ERROR] Error in parsing the input {input} : {e}")

  @classmethod
  def from_properties(cls, type: str, conditions: list, result: str | None = None):
    """Alternative constructor to initialize properties directly."""
    obj = cls.__new__(cls)
    obj.type = type
    obj.conditions = conditions
    obj.result = result
    return obj
  
  def is_valid(self):
    return 'type' in self.__dict__

  def display(self):
    if (not self.is_valid()):
      print("[ERROR] Cannot display invalid Query")
    else:
      print(f"Type : {self.type}")
      print(f"Conditions : {self.conditions}")
      if (self.result is not None) : print(f"Result : {self.result}")

In [ ]:
class Node:
  def __init__ (self, conditions : list = [], result : str = None):
    self.rule_conditions = conditions
    self.rule_result = result
    self.true_node = None
    self.false_node = None

  def check_rule(self, input: list):
    # If self.rule_conditions = [] (empty list) => that means if TRUE (always True)
    upper_input = [data.upper() for data in input]
    for cond in self.rule_conditions:
      if (cond.upper() not in upper_input):
        return False
    return True

  def display_node(self):
    print(f"Rule Conditions : {self.rule_conditions}")
    print(f"Rule Result : {self.rule_result}" )
    print(f"True Node : {self.true_node is not None}")
    print(f"False Node : {self.false_node is not None}")

  def get_final_result(self, conditions: list, newest_result: str):
    if self.check_rule(conditions):
      remaining_cond = [cond for cond in conditions if cond not in self.rule_conditions]
      newest_result = self.rule_result
      if self.true_node is not None:
        return self.true_node.get_final_result(remaining_cond, newest_result)
    else:
      if self.false_node is not None:
        return self.false_node.get_final_result(conditions, newest_result)
    
    return newest_result

  def add_node(self, conditions: list, result: str):
    if self.check_rule(conditions):
      remaining_cond = [cond for cond in conditions if cond not in self.rule_conditions]
      if self.true_node is None:
        self.true_node = Node(remaining_cond, result)
      else:
        self.true_node.add_node(remaining_cond, result)
    else:
      if self.false_node is None:
        self.false_node = Node(conditions, result)
      else:
        self.false_node.add_node(conditions, result)

  def print_subtree(self, prefix: str = "  |"):
    rule_conditions_str = "TRUE" if not self.rule_conditions else self.rule_conditions
    print(f"{rule_conditions_str} -> {self.rule_result}")
      
    if self.true_node != None:
      prefix1 = prefix + "  |"
      print(f"{prefix1}- true node: ", end="")
      if self.false_node == None:
        prefix2 = prefix + "   "
      else:
        prefix2 = prefix + "  |"
      self.true_node.print_subtree(prefix2)
    
    if self.false_node != None:
      prefix1 = prefix + "  |"
      print(f"{prefix1}- false node: ", end="")
      prefix2 = prefix + "   "
      self.false_node.print_subtree(prefix2)
      
  def save_subtree(self, file, prefix: str = ""):
      # Write the current node's details
      rule_conditions_str = "TRUE" if not self.rule_conditions else self.rule_conditions
      file.write(f"{rule_conditions_str} -> {self.rule_result}\n")
      
      # Update the prefix for child nodes
      if self.true_node != None:
          prefix1 = prefix + "  |"
          # Write the "true node" label and recursively save the true node
          file.write(f"{prefix1}- true node: ")
          if self.false_node == None:
            prefix2 = prefix + "   "
          else:
            prefix2 = prefix + "  |"
          self.true_node.save_subtree(file, prefix2)
      
      if self.false_node != None:
          prefix1 = prefix + "  |"
          # Write the "false node" label and recursively save the false node
          file.write(f"{prefix1}- false node: ")
          prefix2 = prefix + "   "
          self.false_node.save_subtree(file, prefix2)
          


In [ ]:
class RDR:
  tree : Node | None

  def __init__ (self):
    self.tree = None

  def execute_query(self, query: Query) -> str | None:
    # Additional handling
    if (not query.is_valid()):
      print(f"[ERROR] Cannot execute invalid query")
      return None
    
    if (query.type == "ask"):
      return self.get_result(query)
    
    else: # query.type == "add"
      self.add_rule(query)
    
  def add_rule(self, query: Query):
    conditions = query.conditions
    result = query.result

    if self.tree is None:
      self.tree = Node(conditions, result)
    else:
      self.tree.add_node(conditions, result)

  def get_result(self, query: Query) -> str | None :
    conditions = query.conditions
    result = "Not mapped"

    if self.tree is None:
      print("Tree is empty")
    else:
      result = self.tree.get_final_result(conditions, result)
    return result

  def print_tree(self):
    if self.tree is None:
      print("Tree is empty")
    else:
      print("Printing the RDR tree:")
      self.tree.print_subtree("")
      
  def save_txt_tree(self, filename: str):
      if self.tree is None:
          print("Tree is empty")
      else:
          print(f"Saving tree to {filename}.txt")
          with open(os.path.join(os.getcwd(), '../test', 'txt', filename + ".txt"), "w") as file:
              self.tree.save_subtree(file)
              
  def load_txt_tree(self):
    file_name = input("Enter the file name to load: ").strip()
    
    # Construct the full path to the text file in the 'test/txt' directory
    file_path = os.path.join(os.getcwd(), '../test', 'txt', file_name + ".txt")
        
    # Check if the file exists
    while not os.path.exists(file_path):
        print(f"[ERROR] File {file_path} not found")
        file_name = input("Enter the file name: ").strip()
        file_path = os.path.join(os.getcwd(), '../test', 'txt', file_name + ".txt")

    print(f"\nLoading tree from {file_path}\n")
        
    # Open the file and process it
    with open(file_path, "r") as file:
      lines = file.readlines()  # Read all lines
      self.tree = self.load_txt_subtree(lines, 0, len(lines), 0)
  
  
    
  def load_txt_subtree(self, lines: list[str], start_index: int, end_index: int, depth: int) -> tuple[Node, int]:

    # Extract the current line and determine the level of indentation
    current_line = lines[start_index]

    # Remove unnecessary substr
    list_to_replace = ["|-", "|", "true node:", "false node:"]
    for str_to_replace in list_to_replace:
        current_line = current_line.replace(str_to_replace, "")
    current_line = current_line.strip()
    
    # Extract conditions and result from the line
    rule_conditions_str, rule_result = current_line.split("->")
    rule_conditions_str = rule_conditions_str.strip()
    rule_result = rule_result.strip()

    if rule_conditions_str == "TRUE":
        rule_conditions = []
    else:
        # Convert conditions string to list
        try:
            rule_conditions = re.findall(r"'(.*?)'", rule_conditions_str)
            # Debug: print the parsed conditions
        except Exception as e:
            rule_conditions = []

    current_node = Node(rule_conditions, rule_result)

    next_depth = depth + 1
    true_node_index_range = [None, None]
    false_node_index_range = [None, None]

    def count_before_character(s, char):
      if char in s:
          return s.index(char)
      else:
          return 0 
      
    # Search for children node
    for i in range(start_index, end_index):
      line = lines[i]
      if (count_before_character(line, "-")//3 == next_depth):
          if ("true node:" in line):
            true_node_index_range[0] = i
          elif ("false node:" in line):
            false_node_index_range[0] = i

    if (true_node_index_range[0] is not None):
      true_node_index_range[1] = false_node_index_range[0] if (false_node_index_range[0] is not None) else end_index
      current_node.true_node = self.load_txt_subtree(lines, true_node_index_range[0], true_node_index_range[1], next_depth)
    if (false_node_index_range[0] is not None):
      false_node_index_range[1] = end_index 
      current_node.false_node = self.load_txt_subtree(lines, false_node_index_range[0], false_node_index_range[1], next_depth)
    
    return current_node
  

In [ ]:
# Main function to run the RDR system
def main():
    rdr_system = RDR()

    print("Welcome to the RDR system!")
    print("You can enter queries in the following formats:")
    print(" - Ask: [CONDITIONS] ? (Example: A, B, C ?)")
    print(" - Add: [CONDITIONS] > [RESULT] (Example: A, B, C > D)\n")
    
    # Add a message to prompt the user to load or create a new tree
    print("Would you like to load an existing RDR tree or create a new one?")
    print("1. Load existing tree")
    print("2. Create new tree")
    input_option = input("Enter load or create option: ").strip()
    
    while input_option.lower() not in ["load", "create"]:
        print("\nInvalid option. Please enter 'load' or 'create'.")
        input_option = input("Enter load or create option: ").strip()
    
    if input_option == "load":
        rdr_system.load_txt_tree()
    elif input_option == "create":
        pass

    while True:
        try:
            message = """
            type 'exit' to quit\n
            type 'print' to output the RDR structure\n
            type 'save' to save the RDR structure\n
            Enter a query: 
            """
            user_input = input(message).strip()

            if user_input.lower() == 'exit':
                print("Exiting the system. Goodbye!")
                break
            if user_input.lower() == 'print':
                rdr_system.print_tree()
                continue
            if user_input.lower() == 'save':
                filename = input("Enter the file name to save: ").strip()
                rdr_system.save_txt_tree(filename)
                continue
            try:
                query = Query(user_input)
            except ValueError as e:
                print(f"[ERROR] Failed to parse query: {e}")
                continue  # Skip to tsshe next input attempt

            if query.type == "ask":
                result = rdr_system.execute_query(query)
                print(f"Result for query: {result}")
            elif query.type == "add":
                rdr_system.execute_query(query)
                print(f"Rule added: {user_input}")
            else:
                print("[ERROR] Invalid query type.")


        except KeyboardInterrupt:
            print("\nProcess interrupted. Exiting...")
            break

# Start the main loop
if __name__ == "__main__":
    main()

Welcome to the RDR system!
You can enter queries in the following formats:
 - Ask: [CONDITIONS] ? (Example: A, B, C ?)
 - Add: [CONDITIONS] > [RESULT] (Example: A, B, C > D)

Would you like to load an existing RDR tree or create a new one?
1. Load existing tree
2. Create new tree

Loading tree from c:\Users\melvi\OneDrive - Institut Teknologi Bandung\SEMESTER 7\IF4070 Representasi Pengetahuan dan Penalaran\Tubes 1\IF4070_Tugas2\src\../test\txt\insurance_claim_knowledge.txt

Printing the RDR tree:
['collision', 'at-fault'] -> requires inspection
  |- true node: ['hit-and-run'] -> apply uninsured motorist coverage
  |  |- false node: ['major damage'] -> requires inspection
  |     |- false node: ['minor damage'] -> apply repair estimate
  |        |- false node: ['uninsured motorist'] -> deny uninsured motorist coverage
  |- false node: ['collision', 'not at-fault'] -> check third-party insurance
     |- true node: ['major damage'] -> check third-party insurance
     |  |- false node: ['